In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim 
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
movie= pd.read_csv('data/movies_metadata.csv', encoding='utf-8')

In [4]:
book = pd.read_csv('data/books.csv', encoding='utf-8')

In [20]:
data = pd.read_csv('data/movie_book.csv',encoding='utf-8')

In [25]:
data=data[data['overview'].notnull()].reset_index(drop=True)

In [26]:
data.shape

(51060, 9)

In [5]:
meta = pd.read_csv('data/movies_metadata.csv', low_memory=False)
meta = meta[meta['original_title'].notnull()].reset_index(drop=True)
meta = meta[meta['overview'].notnull()].reset_index(drop=True)

In [6]:
meta.head(10)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
5,False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,1995-12-15,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0
6,False,NaN,58000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,...,1995-12-15,0.0,127.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,False,6.2,141.0
7,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",...,1995-12-22,0.0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,Tom and Huck,False,5.4,45.0
8,False,NaN,35000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,...,1995-12-22,64350171.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,False,5.5,174.0
9,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,1995-11-16,352194034.0,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0


In [7]:
from nltk.corpus import stopwords 
from tqdm.notebook import tqdm
from nltk.tokenize import word_tokenize, sent_tokenize
# english 형태로 불용어를 뽑아줍니다. 
stop_words = set(stopwords.words('english')) 

In [9]:
import re

In [10]:



overview = []
for words in tqdm(meta['overview']):
    # word_tokenize을 통해서 Tokenizer를 진행해줍니다. 
    word_tokens = word_tokenize(words)
    # re.sub('[^a-z0-9]+', ' ', str(word_tokens)) 명령어를 통해서 특수문자를 모두 제거해줍니다. 
    sentence = re.sub('[^A-Za-z0-9]+', ' ', str(word_tokens))
    # 양끝에 존재하는 공백을 제거해줍니다. 
    sentence = sentence.strip()

    # 문장의 Tokenizer를 진행해줍니다. 
    sentence_tokens = word_tokenize(sentence)
    result = ''
    for token in sentence_tokens: 
        if token not in stop_words:
            result += ' ' + token 
    result = result.strip().lower()
    overview.append(result)

  0%|          | 0/44512 [00:00<?, ?it/s]

In [11]:
meta['pre_overview'] = overview

In [13]:
from gensim.models import doc2vec

In [14]:
doc_vectorizer = doc2vec.Doc2Vec(
    dm=0,            # PV-DBOW / default 1
    dbow_words=1,    # w2v simultaneous with DBOW d2v / default 0
    window=10,        # distance between the predicted word and context words
    size=100,        # vector size
    alpha=0.025,     # learning-rate
    seed=1234,
    min_count=5,    # ignore with freq lower
    min_alpha=0.025, # min learning-rate
    workers=4,   # multi cpu
    hs = 1,          # hierar chical softmax / default 0
    negative = 10   # negative sampling / default 5
)

In [15]:
from collections import namedtuple

# Doc2Vec 입력형태를 맞춰줍니다. 
# TaggedDocument 형태로 ('영화 내용', '제목') 을 받습니다. 
## 이후, TaggedDocument에 모든 문서에 대해서 '영화 내용', '제목'을 넣어줍니다. 
agg = meta[['original_title', 'pre_overview']]
TaggedDocument = namedtuple('TaggedDocument', 'words tags')
tagged_train_docs = [TaggedDocument((c), [d]) for d, c in agg[['original_title', 'pre_overview']].values]

In [16]:
doc_vectorizer.build_vocab(tagged_train_docs)
print(str(doc_vectorizer))

Doc2Vec(dbow+w,d100,n10,hs,w10,mc5,s0.001,t4)


In [17]:
from time import time

start = time()

for epoch in tqdm(range(30)):
    # Doc2Vec 학습을 진행하는데 Learning rate를 계속 감소해주면서 학습을 진행해줍니다. 
    doc_vectorizer.train(tagged_train_docs, total_examples=doc_vectorizer.corpus_count, epochs=doc_vectorizer.iter)
    doc_vectorizer.alpha -= 0.002 # decrease the learning rate
    doc_vectorizer.min_alpha = doc_vectorizer.alpha # fix the learning rate, no decay

end = time()
print("During Time: {}".format(end-start))

  0%|          | 0/30 [00:00<?, ?it/s]

During Time: 1509.0142221450806


In [1]:
doc_vectorizer='./clust/model/my_only.model'

In [2]:
doc_vectorizer.docvecs.most_similar('Toy Story', topn=20)

AttributeError: 'str' object has no attribute 'docvecs'

In [19]:
doc_vectorizer.docvecs.most_similar('Harry Potter and the Deathly Hallows: Part 1', topn=20)

[('Handsome Harry', 0.8001950979232788),
 ('Blood River', 0.746220588684082),
 ('Dragonlance: Dragons Of Autumn Twilight', 0.7406488656997681),
 ('Fear and Desire', 0.7231996059417725),
 ('Hatchet II', 0.7199779748916626),
 ('The Great Ecstasy of Robert Carmichael', 0.7176628112792969),
 ('Orwell Rolls in His Grave', 0.7174854278564453),
 ('Classe tous risques', 0.7173769474029541),
 ('Glue', 0.7142724990844727),
 ('밤과 낮', 0.7094244956970215),
 ('Just Like Us', 0.7087857723236084),
 ('$ Dollars', 0.7087770104408264),
 ('我知女人心', 0.7060498595237732),
 ('Der Räuber', 0.7056949734687805),
 ('Never Let Me Go', 0.7054032683372498),
 ('Bart Got a Room', 0.7008174657821655),
 ('The Black Rose', 0.6981192231178284),
 ('Just Go with It', 0.6980278491973877),
 ('Vampires Suck', 0.6974518299102783),
 ('Cirque du Soleil: Varekai', 0.6964730024337769)]